# 텍스트 분석 수행 프로세스

1. 텍스트 사전 준비작업(텍스트 전처리)
2. 피처 벡터화/추출
3. ML 모델 수립 및 학습/예측/평가

## 1. 텍스트 사전 준비작업(텍스트 전처리) - 텍스트 정규화
- 클렌징
- 토큰화
- 필터링/스톱 워드 제거/철자 수정
- Stemming
- Lemmatizationm


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

### 1.1 클렌징 

텍스트에서 분석에 오히려 방해가 되는 불필요한 문자, 기호 등을 사전에 제거. HTML, XML 태그나 특정 기호 등을 사전에 제거

### 1.2 텍스트 토큰화 

- 문장 토큰화(sentence tokenization)  
문장의 마침표, 개행문자(\n) 등 문장의 마지막을 뜻하는 기호에 따라 분리하는 것이 일반적. 정규표현식에 따른 문장토큰화 가능. 
sent_tokenize() 이용해 토큰화 수행  
반환하는 것은 각각의 문장으로 구성된 list 객체. 반환된 list 객체는 5개의 문장으로 된 문자열을 가지고 있다. 

- 단어 토튼화(word tokenization)  
문장을 단어로 토큰화 하는 것. 콤마, 마침표, 개행문자, 정규 표현식 등  
Bag of Word : 단어의 순서가 중요하지 않은 경우. 문장 토큰화 사용하지 않고 단어 토큰화 사용
문장 토큰화는 각각의 문장이 가지는 시맨틱적 의미가 중요한 경우에 사용

In [6]:
# 텍스트 토큰화 

# 문장 토큰화

from nltk import sent_tokenize

text_sample = 'The Matrix is everywhere. It is all around us. Even now, in this very room.\
    You can see it when you look out your window or when you turn on your television.\
    You can feel it when you go to work, or go to church or pay your taxes.'

sentences = sent_tokenize(text= text_sample)

In [7]:
print(type(sentences), len(sentences))
print(sentences)

<class 'list'> 5
['The Matrix is everywhere.', 'It is all around us.', 'Even now, in this very room.', 'You can see it when you look out your window or when you turn on your television.', 'You can feel it when you go to work, or go to church or pay your taxes.']


In [8]:
# 단어 토큰화

from nltk import word_tokenize

sentence = 'The Matrix is everywhere. It is all around us. Even now, in this very room.'
words = word_tokenize(sentence)

In [9]:
print(type(words), len(words))
print(words)

<class 'list'> 19
['The', 'Matrix', 'is', 'everywhere', '.', 'It', 'is', 'all', 'around', 'us', '.', 'Even', 'now', ',', 'in', 'this', 'very', 'room', '.']


- tokenize_text 함수

word_tokenize, sent_tokenize 를 조합해 문서에 대해서 모든 단어를 토큰화  
문서를 먼저 문장으로 나누고, 개별 문장을 다시 단어로 토큰화 하는 함수 만들기 

In [19]:
from nltk import word_tokenize, sent_tokenize


def tokenize_text(text):
    
    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)
    
    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    
    return word_tokens

# 여러 문장에 대해 문장별 단어 토큰화 수행 
word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 5
[['The', 'Matrix', 'is', 'everywhere', '.'], ['It', 'is', 'all', 'around', 'us', '.'], ['Even', 'now', ',', 'in', 'this', 'very', 'room', '.'], ['You', 'can', 'see', 'it', 'when', 'you', 'look', 'out', 'your', 'window', 'or', 'when', 'you', 'turn', 'on', 'your', 'television', '.'], ['You', 'can', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


5개 문장을 문장별로 먼저 토큰화 했으므로 word_tokens 변수는 5개의 리스트 객체를 내포하는 리스트이다. 내포된 리스트 객체는 각각 문장별로 토큰화된 단어를 요소로 가지고 있다. 

- 문장을 단어별로 하나씩 토큰화 할 경우 문맥적인 의미 무시된다.

- n-gram 문맥적 의미 무시되는 것 해결. 연속된 n 개의 단어를 하나의 토큰화 단위로 분리해 내는것. n 개의 단어 크기 윈도우를 만들어 문장의 처음부터 오른쪽으로 움직이면서 토큰화 수행

- 2-gram ('The', 'Matrix'), ('Matrix', 'is'), ('is', 'everywhere') 연속적으로 2개의 단어들을 순차적으로 이동하면서 단어들을 토큰화


 ### 1.3 스톱워드 제거 

- 스톱워드(stop word)는 분석에 큰 의미가 없는 단어 지칭한다. 
- 가령 영어에서 is, the, a, will 등 문장을 구성하는 필수 문법 요소지만, 문맥적으로 큰 의미가 없는 단어가 이에 해당한다. 
- 이 단어의 경우 문법적인 특성으로 인해 특히 빈번하게 텍스트에 나타나므로 이것들을 사전에 제거하지 않으면 그 빈번함으로 인해 오히려 중요한 단어로 인지될 수 있다. 따라서 이 의미없는 단어를 제거하는 것이 중요한 전처리 작업이다. 

언어별로 이러한 스톱워드가 목록화 되어 있다. NLTK 경우 가장 다양한 언어의 스톱 워드를 제공한다, NLTK 의 스톱워드에는 어떤 것이 있는지 확인해 보겠다.


In [15]:
# NLTK 의 스톱워드 목록 확인. 

# 스톱워드 다운로드. 언어별로 있음

import nltk 
nltk.download('stopwords')

# NLTK 의 english 의 경우 몇 개의 stopwords 가 있는지 알아보고 그 중 20개만 확인

print('영어 stop word 개수: ', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

# 영어의 경우 스톱워드 179 개. 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
영어 stop word 개수:  179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [20]:
# 5 개의 문장별로 단어를 토큰화해 생성된 word_tokens 리스트에 대해서 stopwords 를 필터링 으로 제거
# 분석을 위한 의미있는 단어만 추출하기 

import nltk

stopwords = nltk.corpus.stopwords.words('english')

all_tokens = []

# 위 예제게섯 5개의 문장별로 얻은 word_tokens list 에 대해 스톱워드를 제거하는 반복문
for sentence in word_tokens:

    filtered_words = []
    # 개별 문장별로 토큰화된 문장 list 에 대해 스톱워드를 제거하는 반복문
    for word in sentence:
        # 소문자로 변환
        word = word.lower()
        
        # 토큰화된 개별 단어가 스톱 워드의 단어에 포함되지 않으면 word_tokens 에 추가
        if word not in stopwords:
            filtered_words.append(word)

    all_tokens.append(filtered_words)

# 의미 있는 단어만 추출되었는지 확인
print(all_tokens)

# is, this 와 같은 스톱 워드가 필터링을 통해 제거되었다. 

[['matrix', 'everywhere', '.'], ['around', 'us', '.'], ['even', ',', 'room', '.'], ['see', 'look', 'window', 'turn', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


### 1.4 Stemming 과 Lemmatization

문법적 or 의미적으로 변화하는 단어의 원형을 찾는 것 

#### Lemmatization 
- Stemming 보다 정교하며 의미론적인 기반에서 단어의 원형을 찾는다. 정확한 철자로 된 어근 단어 찾아준다. 시간 오래 걸림  
- WordNetLemmatizer

#### Stemming 
- 원형 단어로 변환 시 일반적인 방법 적용하거나 더 단순화된 방법을 적용해 원래 단어에서 일부 철자가 훼손된 어근 단어를 추출하는 경향이 있음.  
- Porter, Lancaster, Snowball Stemmer

